In [1]:
# !pip install dmba

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.formula.api as sm

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

import matplotlib.pylab as plt
%matplotlib inline

In [3]:
try:
    boston_df = pd.read_csv('BostonHousing.csv')
except:
    print('BostonHousing.csv is not in your pwd')

In [4]:
boston_df.head()

,CRIME,ZONE,INDUST,CHAR RIV,NIT OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST RATIO,LOW STAT,MVALUE,C MVALUE
0,0.00632,18.0,2.31,N,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,No
1,0.02731,0.0,7.07,N,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,No
2,0.02729,0.0,7.07,N,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,Yes
3,0.03237,0.0,2.18,N,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,Yes
4,0.06905,0.0,2.18,N,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,Yes


In [5]:
# 1a.  Create a boston_df data frame by uploading the original data set into Python. Determine 
# and present in this report the data frame dimensions, i.e., number of rows and columns 
# from Python, and briefly explain these numbers. 
nrows = boston_df.shape[0]
ncols = boston_df.shape[1]
print('There are', nrows, 'rows/records and', ncols, 'columns/features in the Boston Housing dataset.')
print('Each row/record represents a home and each column/feature is attribute that the home in this dataset possess.')

There are 506 rows/records and 14 columns/features in the Boston Housing dataset.
Each row/record represents a home and each column/feature is attribute that the home in this dataset possess.


In [6]:
# 1b. Display in Python the column titles and present them in your report. If some of them 
# contain two (or more) words, convert them into one-word titles, and present the modified 
# titles in your report.  
boston_df.columns = boston_df.columns.str.replace(' ', '_')

In [7]:
# 1c. Display in Python column data types and present them in your report. If some of them are 
# listed as “object’, briefly explain that in your report, convert them into dummy variables, 
# and provide in your report the modified list of column titles with dummy variables. 

In [8]:
boston_df.dtypes
# CHAR_RIV, C_MVALUE are both of type object. 

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object

In [9]:
boston_df.CHAR_RIV = boston_df.CHAR_RIV.astype('category')
boston_df.C_MVALUE = boston_df.C_MVALUE.astype('category')

In [10]:
boston_df = pd.get_dummies(boston_df, prefix_sep='_', 
                            drop_first=True)

In [11]:
# 1d. Display in Python the descriptive statistics for all columns in the modified boston_df data 
# frame (after converting to one-word titles and dummy variables). Check if there are 
# missing records (values) in the columns. Present the table with descriptive statistics in 
# your report, and comment about the missing values. You don’t need to comment on the 
# values of outliers (min/max) or their extreme values.

In [12]:
# Display descriptive stats
boston_df.describe()

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806,0.069170,0.166008
std,8.601545,23.322453,6.860353,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104,0.253994,0.372456
min,0.006320,0.000000,0.460000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000,0.000000,0.000000
25%,0.082045,0.000000,5.190000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000,0.000000,0.000000
50%,0.256510,0.000000,9.690000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000,0.000000,0.000000
75%,3.677083,12.500000,18.100000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000,0.000000,0.000000
max,88.976200,100.000000,27.740000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000,1.000000,1.000000


In [13]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CRIME         506 non-null    float64
 1   ZONE          506 non-null    float64
 2   INDUST        506 non-null    float64
 3   NIT_OXIDE     506 non-null    float64
 4   ROOMS         506 non-null    float64
 5   AGE           506 non-null    float64
 6   DISTANCE      506 non-null    float64
 7   RADIAL        506 non-null    int64  
 8   TAX           506 non-null    int64  
 9   ST_RATIO      506 non-null    float64
 10  LOW_STAT      506 non-null    float64
 11  MVALUE        506 non-null    float64
 12  CHAR_RIV_Y    506 non-null    uint8  
 13  C_MVALUE_Yes  506 non-null    uint8  
dtypes: float64(10), int64(2), uint8(2)
memory usage: 48.6 KB


In [14]:
# check for missing values
boston_df.isnull().sum()

CRIME           0
ZONE            0
INDUST          0
NIT_OXIDE       0
ROOMS           0
AGE             0
DISTANCE        0
RADIAL          0
TAX             0
ST_RATIO        0
LOW_STAT        0
MVALUE          0
CHAR_RIV_Y      0
C_MVALUE_Yes    0
dtype: int64

In [15]:
# 2a. Develop in Python outcome and predictor variables, partition the data set (70% for 
# training and 30% for validation partitions), and train the multiple linear regression model 
# using LinearRegression() with the training data set. Identify and display in Python 
# intercept and regression coefficients of this model. Provide these coefficients in your 
# report and present the mathematical equation of this linear regression model.    

In [16]:
predictors = ['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']
outcome = 'MVALUE'
 
X = boston_df[predictors]
y = boston_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [17]:
boston_lm = LinearRegression()
boston_lm.fit(train_X, train_y)

print('Regression Model for Boston Housing Training Set')
print()
print('Intercept: ', np.round(boston_lm.intercept_, 2)) # coefficients
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_lm.coef_, 2)}))

Regression Model for Boston Housing Training Set

Intercept:  48.62
       Predictor  Coefficient
0          CRIME        -0.15
1           ZONE        -0.01
2         INDUST         0.13
3      NIT_OXIDE       -17.86
4          ROOMS         0.33
5            AGE        -0.01
6       DISTANCE        -0.66
7         RADIAL         0.22
8            TAX        -0.01
9       ST_RATIO        -0.63
10      LOW_STAT        -0.47
11    CHAR_RIV_Y         2.33
12  C_MVALUE_Yes        12.13


In [18]:
# MVALUE = 48.62 + (-0.15)CRIME + (-0.01)ZONE + (0.13)INDUST + (-17.86)NIT_OXIDE + (0.33)ROOMS + (-0.01)AGE 
# (-0.66)DISTANCE + (0.22)RADIAL + (-0.01)TAX + (-0.63)ST_RATIO + (-0.47)LOW_STAT + (2.33)CHAR_RIV_Y + (12.13)C_MVALUE_Yes

In [19]:
# 2b. Using the multiple regression model, identify in Python predictions for validation and 
# training predictors (valid_X and train_X). Based on these predictions, identify and display 
# in Python R2 and adjusted R2 performance measures for training and validation partitions. 
# Present and compare these performance measures in your report and explain if there is 
# a possibility of overfitting.    

In [20]:
# predictions for validation set.
boston_lm_pred = boston_lm.predict(valid_X)

# training set.
pred_y = boston_lm.predict(train_X)

In [21]:
# Create prediction performance measures for training set.
r2 = round(r2_score(train_y, pred_y),3)
adj_r2 = round(adjusted_r2_score(train_y, pred_y, boston_lm),3)

# Display prediction performance measures for training set.
print('Prediction Performance Measures for Training Set')
print('r2 : ', r2)
print('Adjusted r2 : ', adj_r2)

# Create prediction performance measures for validation set.
r2 = round(r2_score(valid_y, boston_lm_pred),3)
adj_r2 = round(adjusted_r2_score(valid_y, boston_lm_pred, boston_lm),3)

# Display prediction performance measures for validation set.
print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2)
print('adjusted r2 : ', adj_r2)

Prediction Performance Measures for Training Set
r2 :  0.83
Adjusted r2 :  0.824
Prediction Performance Measures for Validation Set
r2 :  0.852
adjusted r2 :  0.838


In [22]:
# 2c. Identify and display in Python the common accuracy measures for training and validation 
# data set (predictions). Provide and compare these accuracy measures in your report and 
# assess again a possibility of overfitting. 

In [23]:
# Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, boston_lm_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.7145
            Mean Absolute Error (MAE) : 2.6931
          Mean Percentage Error (MPE) : -2.7567
Mean Absolute Percentage Error (MAPE) : 13.2197

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.3667
       Root Mean Squared Error (RMSE) : 3.6868
            Mean Absolute Error (MAE) : 2.7428
          Mean Percentage Error (MPE) : -2.9628
Mean Absolute Percentage Error (MAPE) : 13.9356


In [24]:
# 3a. Use the Exhaustive Search algorithm in Python to identify the best predictors for the 
# multiple linear regression model. Based on these predictors, train a new multiple linear 
# regression model using the respective training data set predictors and 70%-30% partition 
# of the data set. Identify and display in Python the intercept and regression coefficients of 
# this model and the common accuracy measures for validation partition. Provide these 
# coefficients in your report and present the mathematical equation of the respective 
# multiple linear regression model.   

In [25]:
# Define train_model() function used in Executive Search
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Executive Search 
def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # Apply negative sign as score is optimized to be 
    # as low as possible in exhaustive_search() function.
    return -adjusted_r2_score(train_y, pred_y, model)

# Create allVariables object with predcitors in train_X,
# i.e., training data set, with 13 predictor columns 
allVariables = train_X.columns

# The exhaustive_search() function consists of 3 arguments:
# - allVariables - list of all variables in training data set,
# - train_model() function that creates a model for a specific 
#    combination of variables,
# - score_model() function that score the model performance using
#     adjusted_r2.
results = exhaustive_search(allVariables, train_model, score_model)

# Create data[] loop process to identify and append the best model 
# for each combination of 1, 2, 3, ..., 13 variables with their 
# respective number of variables (n), adjusted R_squared (r2adj) and
# AIC. 
data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)

pd.set_option('display.width', 100)

# Display the Exhaustive Search results.
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))
 
pd.reset_option('display.width')

     n     r2adj          AIC    AGE  CHAR_RIV_Y  CRIME  C_MVALUE_Yes  DISTANCE  INDUST  LOW_STAT  \
0    1  0.615757  2227.470343  False       False  False          True     False   False     False   
1    2  0.784502  2023.736517  False       False  False          True     False   False      True   
2    3  0.793737  2009.222342  False       False   True          True     False   False      True   
3    4  0.800829  1997.822810  False        True   True          True     False   False      True   
4    5  0.804618  1992.008003  False       False  False          True      True   False      True   
5    6  0.811403  1980.477479  False        True  False          True      True   False      True   
6    7  0.816868  1971.047129  False        True   True          True      True   False      True   
7    8  0.822139  1961.682655  False        True   True          True      True   False      True   
8    9  0.822845  1961.248007  False        True   True          True      True    True    

In [26]:
# Identify predictors and outcome of the regression model. n = 10
predictors_ex = ['CHAR_RIV_Y', 'CRIME', 'C_MVALUE_Yes', 'DISTANCE', 'INDUST', 'LOW_STAT', 'NIT_OXIDE', 'RADIAL', 
                 'ROOMS', 'ST_RATIO', 'TAX']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3).
X = boston_df[predictors_ex]
y = boston_df[outcome]
train_X_ex, valid_X_ex, train_y_ex, valid_y_ex = train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y.
boston_ex = LinearRegression()
boston_ex.fit(train_X_ex, train_y_ex)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Exhaustive Search')
print()
print('Intercept ', np.round(boston_ex.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_ex.coef_, 2)}))

Regression Model for Training Set Using Exhaustive Search

Intercept  48.69
       Predictor  Coefficient
0     CHAR_RIV_Y         2.31
1          CRIME        -0.15
2   C_MVALUE_Yes        11.98
3       DISTANCE        -0.69
4         INDUST         0.13
5       LOW_STAT        -0.48
6      NIT_OXIDE       -18.30
7         RADIAL         0.22
8          ROOMS         0.29
9       ST_RATIO        -0.62
10           TAX        -0.01


In [27]:
# MVALUE = 48.69 + (-0.15)CRIME + (0.13)INDUST + (-18.30)NIT_OXIDE + (0.29)ROOMS + (-0.69)DISTANCE + (0.22)RADIAL + (-0.01)TAX
# + (-0.62)ST_RATIO + (-0.48)LOW_STAT + (2.31)CHAR_RIV_Y + (11.98)C_MVALUE_Yes

In [28]:
# predictions for validation set from exhaustive search 
boston_ex_pred = boston_ex.predict(valid_X_ex)

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - Exhaustive Search feature selection')
regressionSummary(valid_y_ex, boston_ex_pred)

Accuracy Measures for Validation Set - Exhaustive Search feature selection

Regression statistics

                      Mean Error (ME) : 0.3628
       Root Mean Squared Error (RMSE) : 3.6801
            Mean Absolute Error (MAE) : 2.7244
          Mean Percentage Error (MPE) : -2.9382
Mean Absolute Percentage Error (MAPE) : 13.8210


In [29]:
# 3b. Use the Backward Elimination algorithm in Python exactly as discussed in 3a.  Provide the 
# same results in your report as discussed in 3a. Also, explain the differences (if any exists) 
# between the best predictors (number and specific predictors used) in the models in 3a 
# and 3b.   

In [30]:
# Define train_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

# Use backward_elimination() function to identify the
# best_model and best_variables. 
best_model_be, best_variables_be = backward_elimination(train_X.columns, 
                        train_model, score_model, verbose=True)
 
print()
print('Best Variables from Backward Elimination Algorithm')
print(best_variables_be)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=1963.68
Step: score=1962.03, remove AGE
Step: score=1960.30, remove ZONE
Step: score=1958.80, remove ROOMS
Step: score=1958.80, remove None

Best Variables from Backward Elimination Algorithm
['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']


In [31]:
# Develop the multiple linear regression model based
# on the Backward Elimination results.

predictors_be = ['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 
                 'CHAR_RIV_Y', 'C_MVALUE_Yes']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3). 
X = boston_df[predictors_be]
y = boston_df[outcome]
train_X_be, valid_X_be, train_y_be, valid_y_be = train_test_split(X, y, test_size=0.3, random_state=1)

# Train Backwards Elimination Model
boston_be = LinearRegression()
boston_be.fit(train_X_be, train_y_be)

# Display intercept and regression coefficients.
print('Regression Model for Training Set Using Backward Elimination')
print()
print('Intercept ', np.round(boston_be.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_be.coef_, 2)}))

Regression Model for Training Set Using Backward Elimination

Intercept  50.82
      Predictor  Coefficient
0         CRIME        -0.15
1        INDUST         0.13
2     NIT_OXIDE       -18.39
3      DISTANCE        -0.69
4        RADIAL         0.23
5           TAX        -0.01
6      ST_RATIO        -0.63
7      LOW_STAT        -0.49
8    CHAR_RIV_Y         2.34
9  C_MVALUE_Yes        12.19


In [32]:
# MVALUE = 50.82 + (-0.15)CRIME + (0.13)INDUST + (-18.39)NIT_OXIDE + (-0.69)DISTANCE + (0.23)RADIAL + (-0.01)TAX
# + (-0.63)ST_RATIO + (-0.49)LOW_STAT + (2.34)CHAR_RIV_Y + (12.19)C_MVALUE_Yes

In [33]:
# predictions for validation set from exhaustive search 
boston_be_pred = boston_be.predict(valid_X_be)

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - Backward Elimination')
regressionSummary(valid_y_be, boston_be_pred)

Accuracy Measures for Validation Set - Backward Elimination

Regression statistics

                      Mean Error (ME) : 0.3854
       Root Mean Squared Error (RMSE) : 3.7318
            Mean Absolute Error (MAE) : 2.7591
          Mean Percentage Error (MPE) : -2.8698
Mean Absolute Percentage Error (MAPE) : 13.9371


In [34]:
# 3c. Present and compare in your report the common accuracy measures for validation data 
# set of the three linear regression models:  with all predictors based on the Exhaustive 
# Search algorithm and based on Backward Elimination algorithm.  Using the value of RMSE 
# and the number of variables in each model, which model would you recommend using for 
# making predictions in this case? Briefly explain your answer.    

In [38]:
# predictions for validation set.
boston_lm_pred = boston_lm.predict(valid_X)

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Predictors')
regressionSummary(valid_y, boston_lm_pred)
# has 13 predictors
# Could be fitting noise even though error isnt much worse than Exhaustive search
# Not simple enough

Accuracy Measures for Validation Set - All Predictors

Regression statistics

                      Mean Error (ME) : 0.3667
       Root Mean Squared Error (RMSE) : 3.6868
            Mean Absolute Error (MAE) : 2.7428
          Mean Percentage Error (MPE) : -2.9628
Mean Absolute Percentage Error (MAPE) : 13.9356


In [36]:
# predictions for validation set from exhaustive search 
boston_ex_pred = boston_ex.predict(valid_X_ex)

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - Exhaustive Search feature selection')
regressionSummary(valid_y_ex, boston_ex_pred)
# has 11 predictors 
# Best Model
# Pros: least error, simple since less predictors
# Cons: time taken to allocate best feature set

Accuracy Measures for Validation Set - Exhaustive Search feature selection

Regression statistics

                      Mean Error (ME) : 0.3628
       Root Mean Squared Error (RMSE) : 3.6801
            Mean Absolute Error (MAE) : 2.7244
          Mean Percentage Error (MPE) : -2.9382
Mean Absolute Percentage Error (MAPE) : 13.8210


In [37]:
# predictions for validation set from backwards elimination 
boston_be_pred = boston_be.predict(valid_X_be)

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - Backward Elimination')
regressionSummary(valid_y_be, boston_be_pred)
# has 10 predictors
# RMSE is higher than Exhaustive and model trained with all predictors
# Although simpliest model, error is not desired. 

Accuracy Measures for Validation Set - Backward Elimination

Regression statistics

                      Mean Error (ME) : 0.3854
       Root Mean Squared Error (RMSE) : 3.7318
            Mean Absolute Error (MAE) : 2.7591
          Mean Percentage Error (MPE) : -2.8698
Mean Absolute Percentage Error (MAPE) : 13.9371
